In this last file we faced the Darcy-Forchheimer equation coupled with an advection-diffusion equation, namely the following problem:

\begin{equation}
\begin{cases}    
    \nu(\theta)\boldsymbol{u} + \beta \lvert \boldsymbol{u}\rvert \boldsymbol{u} + \nabla p = \boldsymbol{f} \;\; \text{in} \;\; \Omega ,\\ \text{div} \,  \boldsymbol{u} = 0 \;\; \text{in} \;\; \Omega , \\ -\kappa\Delta\theta + (\boldsymbol{u} \cdot \nabla)\theta = g \;\; \text{in} \;\; \Omega, \\
     p = p_D, \quad \theta = \theta_0 \;\; \text{on} \;\; \partial\Omega
\end{cases}
\end{equation}


By considering the previous definitions of the functinal spaces we introduce the weak formulation:

\begin{equation*}
    \text{Find}\;\; (\boldsymbol{u},p, \theta) \in \boldsymbol{X} \times M \times V \;\; \text{s.t.} \;\; \theta = \theta_0 \;\;\text{and}
\end{equation*}

\begin{equation}
    \begin{cases}
    \int_{\Omega} (\nu(\theta)\boldsymbol{u} + \beta|\boldsymbol{u}|\boldsymbol{u}) \cdot \boldsymbol{v} - \int_\Omega p\text{div}\boldsymbol{v} = \int_\Omega \boldsymbol{f} \cdot \boldsymbol{v} -\int_{\partial\Omega} p_D \boldsymbol{v} \cdot \boldsymbol{n} \quad\quad \forall\boldsymbol{v} \in \boldsymbol{X} \\
    \int_\Omega q\text{div}\boldsymbol{u} = 0 \quad\quad \forall q \in M \\
    \kappa\int_\Omega \nabla\theta \cdot \nabla\rho + \int_\Omega (\boldsymbol{u} \cdot \nabla) \theta \rho = \int_\Omega g\rho \quad\quad  \forall \rho \in V
\end{cases}
\end{equation}

and the correspondend discrete formulation:

\begin{equation}
    \text{Find} \ \  (\boldsymbol{u}_h,p_h,\theta_h) \in \boldsymbol{X}_h \times M_h \times V_h \ \ \text{s.t.}
\end{equation}

\begin{equation}
    \begin{cases}
    \int_\Omega (\nu(\theta_h)\boldsymbol{u}_h + \beta|\boldsymbol{u}_h|\boldsymbol{u}_h) \cdot \boldsymbol{v}_h - \int_\Omega p_h\text{div}\boldsymbol{v}_h = \int_\Omega \boldsymbol{f} \cdot \boldsymbol{v}_h -\int_{\partial\Omega} p_D \boldsymbol{v}_h \cdot \boldsymbol{n} \quad\quad \forall\boldsymbol{v}_h \in \boldsymbol{X}_h \\
    \int_\Omega q_h\text{div}\boldsymbol{u}_h = 0 \quad\quad \forall q_h \in M_h \\
    a_h(\theta_h,\rho_h) + b_h(\boldsymbol{u}_h,\theta_h,\rho_h) = l_h(\rho_h) \quad\quad \forall \rho_h \in V_h
    \end{cases}
\end{equation}

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# Data of the problem
beta = Constant(1.2)
k = Constant(0.05)
g = Constant(0)
theta_ex = Expression('(exp((x[0] - 1) / k) - exp(-1 / k)) / (1 - exp(-1 / k)) + '
                     '(exp((x[1] - 1) / k) - exp(-1 / k)) / (1 - exp(-1 / k))', k=k, degree=3)

f= Expression(('(1+exp(-theta_ex)) + beta*sqrt(2) + x[1]',
                   '(1+exp(-theta_ex)) + beta*sqrt(2) + x[0]'), degree=4, theta_ex=theta_ex, beta=beta)
u_exact = Constant((1.0,1.0))
p_exact = Expression('x[0]*x[1]', degree=2)


d = 100
degree = 1

In [ ]:
# Definition of the model function for the viscosity
def coeff(u):
  return 1+exp(-u)

In [ ]:
def DiffAdv_DG(mesh, degree, k, g, u, t_exact):

  # 1. definition of finite element space
  V = FunctionSpace(mesh, 'DG', degree)
  W = FunctionSpace(mesh, 'RT', 2)

  # 2. assembling matrices and vectors
  t = TrialFunction(V)
  v = TestFunction(V)
  n = FacetNormal(mesh)

  gamma = 9.1*degree**2
  h = CellDiameter(mesh)
  h_avg = (h('+') + h('-'))/2
  uh = project(u,W)

  a = (k*dot(grad(t) , grad(v)))*dx \
    - (dot(avg(k*grad(t)) , jump(v,n)))*dS \
    - (dot(avg(k*grad(v)) , jump(t,n)))*dS \
    - (dot(k*grad(v) , n*t))*ds \
    + ((gamma/h_avg)*dot(jump(t,n) , jump(v,n)))*dS \
    + (t*v*gamma/h)*ds \
    - (dot(uh*t, grad(v)))*dx \
    + (dot(avg(uh*t) , jump(v,n)))*dS \
    + (abs(dot(uh('+'),n('+')))/2*dot(jump(t,n) , jump(v,n)))*dS \
    + 0.5*dot(uh,n)*t*v*ds \
    - 0.5*div(uh)*t*v*dx

  L = (g*v)*dx \
    - (dot(k*grad(v) , n*t_exact))*ds \
    + (t_exact*v*gamma/h)*ds \
    - 0.5*(dot(uh,n)*t_exact*v)*ds

  th = Function(V)
  solve(a == L, th)

  return th

In [ ]:
def solve_complete(d, degree, nu, k, beta, u_exact, p_exact, theta_ex, f, g):

    # 1. mesh definition
    mesh = UnitSquareMesh(d, d, 'crossed')

    # 2. definition of a stable couple
    V = FiniteElement('RT', mesh.ufl_cell(), degree)
    Q = FiniteElement('DG', mesh.ufl_cell(), degree-1)
    W = FunctionSpace(mesh, MixedElement([V, Q]))
    n = FacetNormal(mesh)

    # 3. define formulation for initial guess (linear Darcy pb)
    u, p = TrialFunctions(W)
    v, q = TestFunctions(W)
    w_old = Function(W)

    # 4. Compute the starting values for the iterative scheme as solutions of
    #    Darcy problem in cascata with the heat equation
    a_in = (nu * dot(u, v) - q * div(u) - p * div(v)) * dx
    L = dot(f, v) * dx - p_exact * dot(v, n) * ds
    solve(a_in == L, w_old)
    u_old, p_old = w_old.split()

    #    now use the velocity as datum
    t_old = DiffAdv_DG(mesh, degree, k, g, u_old, theta_ex)

    # 4. define and solve problem by fixed-point linearization
    niter = 40
    tolerance = 1e-8

    for i in range(niter):
        # 5. Solving a Darcy problem where the coefficient of u depends on both the velocity and the temperature at the previous iteration
        u, p = TrialFunctions(W)
        w = Function(W)
        a = ((coeff(t_old) + beta*sqrt(inner(u_old,u_old))) * dot(u, v) - q * div(u) - p * div(v)) * dx
        solve(a == L, w) #L is always the same
        u, p = w.split()

        #use u as datum for heat equation
        t = DiffAdv_DG(mesh, degree, k, g, u, theta_ex)

        #computing errors
        error = (errornorm(u, u_old, 'Hdiv') / norm(u, 'Hdiv') +
                 errornorm(p, p_old, 'L2') / norm(p, 'L2') + errornorm(t, t_old, 'H1') / norm(t, 'H1'))
        print(' step={}'.format(i))
        u_old = u
        p_old = p
        t_old = t

        if error < tolerance:
            break

        if i == niter:
          print('max_iter_reached')

    # 6. compute absolute errors
    l2err_p = errornorm(p_exact, p_old, 'L2')
    l2err_u = errornorm(u_exact, u_old, 'L2')
    Hdiverr_u = errornorm(u_exact, u_old, 'Hdiv0')
    l2err_t = errornorm(theta_ex, t_old, 'L2')

    return u_old, p_old, t_old, l2err_p, l2err_u, Hdiverr_u, l2err_t

In [ ]:
print(l2err_t)
print(l2err_p)
print(l2err_u)

In [ ]:
# Convergence test
M_err=np.zeros((4,5))

for i in range(5):
    n = 8*(2**i)
    u, p, t, l2err_p, l2err_u, Hdiverr_u, l2err_t = solve_complete(n, degree, 2, k, beta, u_exact, p_exact, theta_ex, f, g)

    M_err[0,i] = l2err_p
    M_err[1,i] = l2err_u
    M_err[2,i] = Hdiverr_u
    M_err[3,i] = l2err_t

In [ ]:
print(M_err)

In [ ]:
# Rescaling the errors
h = np.array([1/8, 1/16, 1/32, 1/64, 1/128])
h = h/h[0]

for i in range(4):
  M_err[i,:] = M_err[i,:]/M_err[i,0]

In [ ]:
# Plots of the orders of convergence
fig = plt.figure(figsize=(8,6))
axes = fig.add_axes([0.2, 0.2, 0.8, 0.8])

axes.loglog(h, h, 'y',label="Order 1")
axes.loglog(h, h**2, 'r',label="Order 2")
axes.loglog(h, h**(1.5), 'k',label="Order 3/2")
axes.loglog(h, h**3, 'b',label="Order 3")
axes.loglog(h, M_err[0,:], 'm--o',label="$L^2$ err p")
axes.loglog(h, M_err[1,:], 'c--o',label="$L^2$ err u")
axes.loglog(h, M_err[3,:], 'g--o',label="$L^2$ err t")

axes.legend(loc=4)

plt.title("Orders of convergence coupled problem");
plt.xlabel("h")
plt.ylabel("Errors")
plt.legend(loc=4)

plt.show()

In [ ]:
# Orders of convergence
orders =np.empty([3,4])

for j in [1,2,3,4]:
  orders[0,j-1]=math.log(M_err[0,j-1]/M_err[0,j],2)#p
  orders[1,j-1]=math.log(M_err[1,j-1]/M_err[1,j],2)#u
  orders[2,j-1]=math.log(M_err[3,j-1]/M_err[3,j],2)#t

print(orders)